In [331]:
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs
import numpy as np
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)
import json
import pandas as pd

In [332]:
#get credentials
with open("postgre_credentials.json") as file:
    data = json.load(file)

In [333]:
def reset_connection():
    if 'connection' not in locals():
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection,cur;
    else:
        connection.close()
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection, cur;

In [334]:
connection, cur = reset_connection()

In [335]:
print(connection)
print(cur)
#template
# connection = psycopg2.connect(
#     database = data["db_name"], 
#     user = data["db_user"],
#     password = data["db_password"],
#     host = data["db_host"],
#     port = data["db_port"]
# )


<connection object at 0x7fa302f47760; dsn: 'user=yjyj password=xxx dbname=anime_trends host=127.0.0.1 port=5432', closed: 0>
<cursor object at 0x7fa300f42040; closed: 0>


In [336]:
#cur = connection.cursor()

In [337]:
query = "DROP TABLE anime CASCADE"
cur.execute(query)
connection.commit()

In [338]:
query="DROP TABLE show_name CASCADE"
cur.execute(query)
connection.commit()

In [339]:
query="""CREATE TABLE IF NOT EXISTS anime (
anime_id INTEGER PRIMARY KEY,
eng_name TEXT,
type TEXT,
source TEXT,
episode_count INTEGER,
maturity_rating TEXT,
MAL_score DECIMAL,
scored_count INTEGER,
MAL_rank INTEGER,
MAL_popularity INTEGER,
favorite_count INTEGER,
studio_name TEXT,
licensor TEXT,
air_status TEXT,
is_airing BOOLEAN,
year INTEGER,
season TEXT
)
"""

In [340]:
cur.execute(query)


In [341]:
connection.commit()

In [342]:
query="""CREATE TABLE IF NOT EXISTS date (
date_id SERIAL PRIMARY KEY,
year INTEGER,
month INTEGER,
day INTEGER
)
"""
cur.execute(query)
connection.commit()

In [343]:
query="""CREATE TABLE IF NOT EXISTS show_name (
show_name_id SERIAL PRIMARY KEY,
alternate_name TEXT,
anime_id INTEGER REFERENCES anime (anime_id)
)
"""
cur.execute(query)
connection.commit()

In [344]:
query="""CREATE TABLE IF NOT EXISTS genre (
genre_id SERIAL PRIMARY KEY,
genre_name TEXT,
anime_id INTEGER REFERENCES anime (anime_id)
)
"""
cur.execute(query)
connection.commit()

In [345]:
query="""CREATE TABLE IF NOT EXISTS google_trends (
google_trends_id SERIAL PRIMARY KEY,
show_name TEXT REFERENCES show_name(alternate_name),
date INTEGER REFERENCES date(date_id),
popularity_score INT
)
"""
cur.execute(query)
connection.commit()

In [346]:
#Loop through MAL csv file
#insert into anime table first
#split alternative names and loop to insert each show name
#split through genres and insert each genre

In [347]:
def split_column(row, col):
    row_attributes = mal_df.loc[row,col]
    split_row = row_attributes.replace(" ", "").split(",")
    return split_row

In [348]:
def insert_genre(split_row, anime_id):
    if len(split_row)>0:
        for genre in split_row:
            cur.execute("""INSERT INTO genre(genre_name, anime_id)
            VALUES (%s,%s)""",
            (genre, anime_id))
            connection.commit()

In [350]:
def insert_alt_name(split_row, anime_id):
    if len(split_row)>0:
        print(split_row)
        for name in split_row:
            
            cur.execute("""INSERT INTO show_name(alternate_name, anime_id)
            VALUES (%s,%s)""",
            (name, anime_id))
            connection.commit()

In [351]:
def insert_anime_info(row):
    try:
        time = row["premiered"].split(" ")
        if time[0] != "Not" and len(time)==3 and "" not in time:
            year = time[1]
            season = time[0]

            cur.execute("""INSERT INTO anime(anime_id, eng_name, type, source,episode_count,
                        maturity_rating, MAL_score, scored_count, MAL_rank, MAL_popularity,
                        favorite_count, studio_name, licensor, air_status, is_airing, year, season)
                        VALUES (%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                       (row["anime_id"],row["title_english"],row["type"], row["source"], row["episodes"],
                       row["rating"], row["score"], row["scored_by"], row["rank"], row["popularity"],
                       row["favorites"], row["studio"], row["licensor"], row["status"], row["airing"],
                       year, season))
            connection.commit()
        
    except: 
        time = row["aired_string"].split("to")[0].split(" ")
        if time[0] != "Not" and len(time)==3 and "" not in time:
            #print(time)
            year = int(time[2])
            season = seasons[time[0]]
            cur.execute("""INSERT INTO anime(anime_id, eng_name, type, source,episode_count,
                        maturity_rating, MAL_score, scored_count, MAL_rank, MAL_popularity,
                        favorite_count, studio_name, licensor, air_status, is_airing, year, season)
                        VALUES (%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                       (row["anime_id"],row["title_english"],row["type"], row["source"], row["episodes"],
                       row["rating"], row["score"], row["scored_by"], row["rank"], row["popularity"],
                       row["favorites"], row["studio"], row["licensor"], row["status"], row["airing"],
                       year, season))
            connection.commit()

        

    


In [352]:
#test with 1 anime
with open("Anime_List_Path.txt") as file:
    path = file.read()

mal_df = pd.read_csv(path)

In [353]:
#mal_df.head()
mal_df.loc[:,"producer"] = mal_df.loc[:,"producer"].fillna("NA")
mal_df.loc[:,"licensor"] = mal_df.loc[:,"licensor"].fillna("NA")
mal_df.loc[:,"title_english"] = mal_df.loc[:,"title_english"].fillna("NA")
mal_df.loc[:,"rank"] = mal_df.loc[:,"rank"].fillna(0)
mal_df.loc[:,"genre"] = mal_df.loc[:,"genre"].fillna("NA")
mal_df.loc[:,"title_synonyms"] = mal_df.loc[:,"title_synonyms"].fillna("NA")
mal_df.loc[:,"title_japanese"] = mal_df.loc[:,"title_japanese"].fillna("NA")

In [354]:
seasons = {"Jan":"Winter",
          "Feb": "Winter",
          "Mar": "Spring",
          "Apr": "Spring",
          "May": "Spring",
          "Jun": "Summer",
          "Jul": "Summer",
          "Aug": "Summer",
          "Sep": "Fall",
          "Oct": "Fall",
          "Nov": "Fall",
          "Dec": "Winter"}

## Insert MAL info (Except Alternate Names and Genre)

In [355]:
#insert_anime_info(mal_df.iloc[0])
for idx,row in mal_df.iterrows():
    insert_anime_info(mal_df.iloc[idx])

## Insert Genres

In [356]:
#mal_df.loc[:,"anime_id"]

In [357]:
#testing
for idx, row in mal_df.iterrows():
    split_genres = split_column(idx, "genre")
    insert_genre(split_genres, mal_df.loc[idx,"anime_id"])

## Insert Alternate Names

In [358]:
mal_df.columns
print(mal_df.loc[0,"title_synonyms"])
print(mal_df.loc[0,"title_japanese"])

Youko x Boku SS
妖狐×僕SS


In [371]:
connection, cur = reset_connection()

In [373]:
for idx, row in mal_df.iterrows():
    try:
        alt_names = split_column(idx, "title_synonyms")
        japanese_names = split_column(idx,"title_japanese")
        all_names = alt_names + japanese_names
        insert_alt_name(all_names, mal_df.loc[idx,"anime_id"])
    except:
        connection,cur = reset_connection()
        continue


['YoukoxBokuSS', '妖狐×僕SS']
['TheInlandSeaBride', '瀬戸の花嫁']
['ShugoCharaNinenme', 'ShugoChara!SecondYear', 'しゅごキャラ！！どきっ']
['NA', 'プリンセスチュチュ']
['BakumanSeason3', 'バクマン。']
['YumeiroPatissiere', 'YumePati', 'Dream-ColoredPastryChef', 'YumeiroPâtissière', '夢色パティシエール']
['NA', 'ウルトラマニアック']
['ShakugannoShana2', '灼眼のシャナII–Second–']
['NodameCantabileParisChapter', 'NodameCantabilePari-hen', 'のだめカンタービレ巴里編']
['OhranKokoHostClub', 'OuranKoukouHosutobu', 'OuranKoukouHostBu', 'OHSHC', '桜蘭高校ホスト部']
['JunjouRomanticaSecondSeason', 'JunjyouRomanticaII', '純情ロマンチカ2(に)']
['ClassPresidentisaMaid!', '会長はメイド様!']
['Sekai-ichiHatsukoi2', 'Sekai&#039;ichiHatsukoi2', '世界一初恋2']
['TonarinoKaibutsukun', 'TheMonsterNextDoor', 'MyNeighborMonster-kun', 'となりの怪物くん']
['NA', 'BLEACH-ブリーチ-']
['NA', 'ちょびっツ']
['ReachingYou', '君に届け']
['NarutoHurricaneChronicles', 'ナルト-疾風伝']
['Ranma1/2', 'Ranma½NettouHen', 'らんま1/2']
['TigerXDragon', 'とらドラ！']
['BakumanSeason2', 'バクマン。2ndシーズン']
['CampusAlice', 'AliceAcademy', '学園アリス']
['NA', 'スキップ・

In [369]:
connection.close()